# ReAct Agent using Pure Python

<img src="react.png" width=400/>

### Table of Contents

1. **Load LLM**
2. **Define Tools**
3. **Define ReAct Prompt**
4. **Define Agent Class**
5. **Write Loop to Call Agent with Action Results**


In [1]:
import groq

print(groq.__version__)

0.9.0


In [2]:
from dotenv import find_dotenv, load_dotenv
import os

load_dotenv(find_dotenv())

groq_api_key = os.environ["GROQ_API_KEY"]

## 1. Load LLM

* Login to **https://console.groq.com** and create API Key.

### Groq Models

<img src="GroqModels.jpg" width=800/>

* **pip install groq**

In [3]:
from groq import Groq

client = Groq(api_key=groq_api_key)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Hello, How are you?",
        }
    ],
    model="llama-3.1-70b-versatile", # llama3-70b-8192
)

print(chat_completion.choices[0].message.content)

Hello! I'm just a language model, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to help you. How about you? How's your day going? Is there something I can help you with or would you like to chat?


## 2. Define Tools

* **pip install yfinance**

In [4]:
import yfinance as yf

def company_address(ticker: str)-> str:
    """
    Returns company address for input ticker.    
    e.g. company_address: AAPL
    Returns company address for ticker AAPL which is stock ticker for Apple Inc.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    
    return " ".join([info[key] for key in ["address1", "city", "state", "zip", "country"]])

def fulltime_employees(ticker: str)-> int:
    """
    Returns fulltime employees count for input ticker.
    e.g. fulltime_employees: MSFT
    Returns fulltime employees count for ticker MSFT which is stock ticker for Microsoft.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    
    return info["fullTimeEmployees"]

def last_close_price(ticker: str)-> float:
    """
    Returns last close price for input ticker.
    e.g. last_close_price: MSFT
    Returns latest close price for ticker MSFT which is stock ticker for Microsoft.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    
    return info["previousClose"]

def EBITDA(ticker: str)-> float:
    """
    Returns EBITDA for input ticker.
    e.g. EBITDA: AAPL
    Returns EBITDA for ticker AAPL which is stock ticker for Apple.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    
    return info["ebitda"]

def total_debt(ticker: str)-> float:
    """
    Returns total debt for input ticker.
    e.g. total_debt: AAPL
    Returns total debt for ticker AAPL which is stock ticker for Apple.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    
    return info["totalDebt"]

def total_revenue(ticker: str)-> float:
    """
    Returns total revenue for input ticker.
    e.g. total_revenue: MSFT
    Returns total revenue for ticker MSFT which is stock ticker for Microsoft.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    
    return info["totalRevenue"]

def debt_to_equity_ratio(ticker: str)-> float:
    """
    Returns debt to equity ratio for input ticker.
    e.g. debt_to_equity_ratio: AAPL
    Returns debt to equity ratio for ticker AAPL which is stock ticker for Apple.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    
    return info["debtToEquity"]

tools = [company_address, fulltime_employees, last_close_price, EBITDA, total_debt, total_revenue, debt_to_equity_ratio]

In [5]:
tools_with_desc = "\n\n".join([tool.__name__ + tool.__doc__ for tool in tools])

print(tools_with_desc)

company_address
    Returns company address for input ticker.    
    e.g. company_address: AAPL
    Returns company address for ticker AAPL which is stock ticker for Apple Inc.
    

fulltime_employees
    Returns fulltime employees count for input ticker.
    e.g. fulltime_employees: MSFT
    Returns fulltime employees count for ticker MSFT which is stock ticker for Microsoft.
    

last_close_price
    Returns last close price for input ticker.
    e.g. last_close_price: MSFT
    Returns latest close price for ticker MSFT which is stock ticker for Microsoft.
    

EBITDA
    Returns EBITDA for input ticker.
    e.g. EBITDA: AAPL
    Returns EBITDA for ticker AAPL which is stock ticker for Apple.
    

total_debt
    Returns total debt for input ticker.
    e.g. total_debt: AAPL
    Returns total debt for ticker AAPL which is stock ticker for Apple.
    

total_revenue
    Returns total revenue for input ticker.
    e.g. total_revenue: MSFT
    Returns total revenue for ticker MSFT w

## 3. Define ReAct Prompt

In [6]:
system_prompt = f"""
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

{tools_with_desc}

Example session:

Question: What is last close price of Apple stock?
Thought: I should retrieve last close price using last_close_price
Action: last_close_price: AAPL
PAUSE

You will be called again with this:

Observation: 223.96

You then output:

Answer: The last close price of Apple stock is 223.96 USD.
""".strip()

In [7]:
print(system_prompt)

You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

company_address
    Returns company address for input ticker.    
    e.g. company_address: AAPL
    Returns company address for ticker AAPL which is stock ticker for Apple Inc.
    

fulltime_employees
    Returns fulltime employees count for input ticker.
    e.g. fulltime_employees: MSFT
    Returns fulltime employees count for ticker MSFT which is stock ticker for Microsoft.
    

last_close_price
    Returns last close price for input ticker.
    e.g. last_close_price: MSFT
    Returns latest close price for ticker MSFT which is stock ticker for Microsoft.
    

EBITDA
    Returns EBITDA for input ticker.
    e.g. EBITDA: AAPL
    Returns EBITDA

## 4. Define Agent Class

In [8]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = [] ### Chat History
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        result = client.chat.completions.create(
            model="llama-3.1-70b-versatile", #llama3-70b-8192
            messages=self.messages,
            temperature=0
        )
        return result.choices[0].message.content

In [9]:
react_agent = Agent(system_prompt)

In [10]:
result = react_agent("What is EBITDA of nvidia?")

print(result)

Thought: I should retrieve EBITDA using EBITDA action.

Action: EBITDA: NVDA
PAUSE


In [11]:
tool_result = EBITDA("NVDA")

tool_result

49274998784

In [12]:
next_prompt = "Observation: {}".format(tool_result)

next_prompt

'Observation: 49274998784'

In [13]:
react_agent(next_prompt)

'Answer: The EBITDA of NVIDIA is 49.275 billion USD.'

In [14]:
result = react_agent("What is EBITDA and debt-to-equity ratio of Google?")

print(result)

Thought: I should retrieve EBITDA and debt-to-equity ratio using EBITDA and debt_to_equity_ratio actions.

Action: EBITDA: GOOGL
PAUSE


In [15]:
tool_result = EBITDA("GOOGL")
next_prompt = "Observation: {}".format(tool_result)
result = react_agent(next_prompt)

print(result)

Thought: I have the EBITDA, now I need the debt-to-equity ratio.

Action: debt_to_equity_ratio: GOOGL
PAUSE


In [16]:
tool_result = debt_to_equity_ratio("GOOGL")
next_prompt = "Observation: {}".format(tool_result)
result = react_agent(next_prompt)

print(result)

Answer: The EBITDA of Google is 115.478 billion USD and the debt-to-equity ratio is 8.295.


In [17]:
result = react_agent("Compare total revenue of Apple and Nvidia.")

print(result)

Thought: I should retrieve total revenue of Apple and Nvidia using total_revenue action.

Action: total_revenue: AAPL
PAUSE


In [18]:
tool_result = total_revenue("AAPL")
next_prompt = "Observation: {}".format(tool_result)
result = react_agent(next_prompt)

print(result)

Thought: I have the total revenue of Apple, now I need the total revenue of Nvidia.

Action: total_revenue: NVDA
PAUSE


In [19]:
tool_result = total_revenue("NVDA")
next_prompt = "Observation: {}".format(tool_result)
result = react_agent(next_prompt)

print(result)

Answer: The total revenue of Apple is 381.623 billion USD and the total revenue of Nvidia is 79.774 billion USD.


In [20]:
total_revenue("NVDA") / 1e9, total_revenue("AAPL") / 1e9

(79.773999104, 381.62300928)

## 5. Write Loop to Call Agent with Action Results

In [21]:
known_actions = dict([(tool.__name__, tool) for tool in tools])

known_actions

{'company_address': <function __main__.company_address(ticker: str) -> str>,
 'fulltime_employees': <function __main__.fulltime_employees(ticker: str) -> int>,
 'last_close_price': <function __main__.last_close_price(ticker: str) -> float>,
 'EBITDA': <function __main__.EBITDA(ticker: str) -> float>,
 'total_debt': <function __main__.total_debt(ticker: str) -> float>,
 'total_revenue': <function __main__.total_revenue(ticker: str) -> float>,
 'debt_to_equity_ratio': <function __main__.debt_to_equity_ratio(ticker: str) -> float>}

In [22]:
import re

action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to selection action

def query(question, max_turns=5):
    i = 0
    react_agent = Agent(system_prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = react_agent(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a) ]
        
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print("=> Running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [23]:
query("Compare total revenue of Apple and Nvidia.")

Thought: I need to compare the total revenue of Apple and Nvidia, so I should retrieve the total revenue for both companies using total_revenue action.

Action: total_revenue: AAPL
PAUSE
=> Running total_revenue AAPL
Observation: 381623009280
Thought: I have the total revenue of Apple, now I need to get the total revenue of Nvidia.

Action: total_revenue: NVDA
PAUSE
=> Running total_revenue NVDA
Observation: 79773999104
Thought: I now have the total revenue of both Apple and Nvidia, so I can compare them.

Answer: The total revenue of Apple is 381623009280, and the total revenue of Nvidia is 79773999104. Apple's total revenue is significantly higher than Nvidia's.


In [24]:
query("What is EBITDA and debt-to-equity ratio of Google?")

Thought: I should retrieve EBITDA and debt-to-equity ratio using EBITDA and debt_to_equity_ratio actions.

Action: EBITDA: GOOGL
PAUSE
=> Running EBITDA GOOGL
Observation: 115478003712
Thought: I have the EBITDA, now I need to get the debt-to-equity ratio.

Action: debt_to_equity_ratio: GOOGL
PAUSE
=> Running debt_to_equity_ratio GOOGL
Observation: 8.295
Thought: I have both EBITDA and debt-to-equity ratio, now I can provide the answer.

Answer: The EBITDA of Google is 115478003712 and the debt-to-equity ratio is 8.295.


## Summary

In this video, I explained how to create **ReAct Agent** using pure **Python**. We did not use any LLM apps building frameworks like **LangChain** or **LlamaIndex**. To build agent, we used **Open Source LLM (LlaMa-3.1-70B)** available through **Groq API**.